In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
pd.set_option('display.max_columns', 200)

In [228]:
path = 'C:/Users/wjssm/Desktop/kaggle/competition/Elo_Merchant_Category_Recommendation/data/'
transaction = pd.read_pickle(path + 'merged_transaction.pkl')
transaction.head()

MemoryError: 

In [187]:
train = pd.read_csv(path + 'train.csv')
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [219]:
train['target_out'] = 0
train.loc[train.target<-33, 'target_out'] =1
train.head()

,first_active_month,feature_1,feature_2,feature_3,target,authorized_flag_new,installments_new,month_lag_new,purchase_amount_new,target_out
card_id,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,1,1,1,1,0
C_ID_3d0044924f,2017-01,4,1,0,0.392913,1,1,1,1,0
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,1,0,1,0,0
C_ID_186d6a6901,2017-09,4,3,0,0.142495,1,1,1,1,0
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,1,1,1,1,0


In [168]:
#transaction data 중 train data에 있는 데이터만 뽑음
train_transaction = transaction.loc[transaction['card_id'].isin(train['card_id'].tolist()),:]
train_transaction.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,target_out
0,1,C_ID_00007093c1,244,N,2,C,705,M_ID_69423b34e4,-12,0.037176,2017-02-14 14:00:43,3.0,2,33,0
1,1,C_ID_00007093c1,244,N,1,B,705,M_ID_a33355a1b7,-12,-0.467415,2017-02-14 15:47:45,3.0,2,33,0
2,1,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-12,-0.572600,2017-02-16 15:37:58,3.0,2,19,0
3,1,C_ID_00007093c1,244,N,3,C,705,M_ID_a33355a1b7,-12,-0.331575,2017-02-20 12:19:01,3.0,2,33,0
4,1,C_ID_00007093c1,244,N,1,B,307,M_ID_9400cf2342,-11,-0.483944,2017-03-03 00:24:15,3.0,2,19,0


In [171]:
train_transaction.shape

(19249694, 15)

In [172]:
train_transaction['target_out'] = 0
train_transaction.loc[train_transaction['card_id'].isin(train.loc[train.target_out == 1, 'card_id'].tolist()),'target_out'] = 1
train_transaction['target_out'].value_counts()

0    19062128
1      187566
Name: target_out, dtype: int64

In [173]:
187566/19062128

0.009839719888566481

In [175]:
train_transaction.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id', 'target_out'],
      dtype='object')

In [174]:
train_transaction.loc[train_transaction.authorized_flag == 'Y', 'authorized_flag'] = 1
train_transaction.loc[train_transaction.authorized_flag == 'N', 'authorized_flag'] = 0

C:\Users\wjssm\Anaconda3\lib\site-packages\pandas\core\ops.py:1164: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: invalid type comparison

In [177]:
num_cols = ['authorized_flag', 'installments','month_lag','purchase_amount']

for col in num_cols :
    print('--------')
    print(col)
    print('전체 평균',train_transaction[col].mean())
    print(train_transaction.groupby(['target_out'])[col].mean())

--------
authorized_flag
전체 평균 0.918862918028723
target_out
0    0.919540
1    0.850053
Name: authorized_flag, dtype: float64
--------
installments
전체 평균 0.6501106978635608
target_out
0    0.646445
1    1.022696
Name: installments, dtype: float64
--------
month_lag
전체 평균 -4.103993393349525
target_out
0   -4.104606
1   -4.041740
Name: month_lag, dtype: float64
--------
purchase_amount
전체 평균 0.11632920337419776
target_out
0    0.116230
1    0.126445
Name: purchase_amount, dtype: float64


In [189]:
col

'purchase_amount'

In [196]:
train_transaction.loc[train_transaction[col] > train_transaction[col].mean(), 'card_id'].unique()

array(['C_ID_00007093c1', 'C_ID_0001506ef0', 'C_ID_000183fdda', ...,
       'C_ID_fffe78b232', 'C_ID_fffea6de74', 'C_ID_fffeed3a89'],
      dtype=object)

In [199]:
train.set_index('card_id',inplace=True)

In [200]:
card_id_list = train_transaction.loc[train_transaction[col] > train_transaction[col].mean(), 'card_id'].unique()

In [203]:
train.head()

,first_active_month,feature_1,feature_2,feature_3,target
card_id,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283
C_ID_3d0044924f,2017-01,4,1,0,0.392913
C_ID_d639edf6cd,2016-08,2,2,0,0.688056
C_ID_186d6a6901,2017-09,4,3,0,0.142495
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749


In [214]:
for col in num_cols :
    train[col+'_new'] = 0
    card_id_list = train_transaction.loc[train_transaction[col] > train_transaction[col].mean(), 'card_id'].unique()
    train.loc[card_id_list,col+'_new'] = 1
        

In [215]:
train.head()

,first_active_month,feature_1,feature_2,feature_3,target,authorized_flag_new,installments_new,month_lag_new,purchase_amount_new
card_id,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,1,1,1,1
C_ID_3d0044924f,2017-01,4,1,0,0.392913,1,1,1,1
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,1,0,1,0
C_ID_186d6a6901,2017-09,4,3,0,0.142495,1,1,1,1
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,1,1,1,1


In [184]:
train_transaction.groupby(['card_id'])['authorized_flag'].mean()

card_id
C_ID_00007093c1    0.768212
C_ID_0001506ef0    0.941176
C_ID_000183fdda    0.954839
C_ID_00027503e2    0.571429
C_ID_0002c7c2c1    0.876712
C_ID_0003be3c83    0.858757
C_ID_00042d509c    0.312500
C_ID_0004888ddd    0.883721
C_ID_0004b68c49    1.000000
C_ID_0004c2a5ab    0.960000
C_ID_0005b2f279    1.000000
C_ID_0005b5804f    1.000000
C_ID_0005f16cc8    0.891026
C_ID_0006152db8    0.886364
C_ID_000616f4a8    0.953488
C_ID_000664aa02    1.000000
C_ID_00079fec55    0.823529
C_ID_0007a60a33    0.656250
C_ID_0007fe3157    0.950495
C_ID_000844d8f1    0.975610
C_ID_000950bc78    1.000000
C_ID_00099cb909    0.916667
C_ID_0009ae74c1    0.903226
C_ID_0009d25138    0.923077
C_ID_0009ef1dc2    1.000000
C_ID_000a6044e9    0.913043
C_ID_000a9864df    1.000000
C_ID_000acab1f6    0.969697
C_ID_000b13135b    0.836538
C_ID_000b503737    0.895000
                     ...   
C_ID_fffa2280c9    0.840000
C_ID_fffa362304    0.800000
C_ID_fffa99d1da    0.987500
C_ID_fffadd3551    0.935484
C_ID_fffaed4

In [178]:
train_transaction.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id', 'target_out'],
      dtype='object')

In [183]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,target_out
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,0
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,0


In [180]:
%who

KFold	 LGBMRegressor	 a	 cat_cols	 col	 cv	 df_cv	 df_train	 f1_score	 
i	 idx_cv	 idx_train	 ip	 jupyternotify	 lgb	 mode	 np	 num_cols	 
path	 pd	 plt	 stats	 test	 test2	 time	 train	 train_transaction	 
x_cv	 x_test	 x_train	 y_cv	 y_cv_pred	 y_test	 y_train	 y_train_pred	 


In [158]:
from scipy import stats
def mode(x):
    return stats.mode(x)[1][0]

In [179]:
cat_cols = ['city_id', 'category_1','category_3','merchant_category_id','merchant_id','category_2','state_id','subsector_id']

for col in cat_cols :
    print('--------')
    print(col)
    print('전체 mode',train_transaction[col].mode())
    print(train_transaction.groupby(['target_out'])[col].value_counts())

--------
city_id
전체 mode 0    69
dtype: int64
target_out  city_id
0            69        3175659
            -1         1553675
             19         750381
             158        731042
             17         623060
             331        467874
             143        439694
             88         427917
             137        393203
             333        359891
             87         301490
             212        270349
             57         249840
             233        234733
             25         234452
             149        233223
             160        214683
             117        211213
             277        165664
             213        164058
             286        153476
             76         142800
             107        138004
             4          135923
             291        135011
             20         134447
             53         134108
             140        131301
             344        115800
             161        111115
    

전체 mode 0    9
dtype: int64
target_out  state_id
0            9          6960087
            -1          1668546
             16         1407334
             15         1366491
             5          1084622
             19          790763
             20          732814
             22          623060
             21          621110
             4           558022
             11          423568
             7           420894
             13          415226
             3           356998
             12          257318
             18          256119
             2           247442
             1           165579
             10          142570
             24          134378
             23          131761
             14          114045
             8           102621
             17           76324
             6             4436
1            9            56333
            -1            37897
             15           14351
             16           12694
             22        

In [ ]:
transaction.groupby(['target_out'])['authorized_flag'].mean()

In [48]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

# training

In [220]:
cv = KFold(5, shuffle=True, random_state=0)

In [221]:
lgb = LGBMRegressor()

In [123]:
path = 'C:/Users/wjssm/Desktop/kaggle/competition/Elo_Merchant_Category_Recommendation/data/'
test = pd.read_csv(path + 'testFE_0213.csv')
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,first_active,first_active_year,first_active_quarter,first_active_weekofyear,first_active_dayofweek,first_active_elapsed_time_from_trade,first_active_elapsed_time_from_today,first_active_total_day,days_feature1_trade,days_feature2_trade,days_feature3_trade,days_feature1_trade_ratio,days_feature2_trade_ratio,days_feature3_trade_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var
0,4.0,C_ID_0ab67a22ab,3,3,1,2017-04-01,2017.0,2.0,13.0,5.0,394.0,682.0,17257.0,1182.0,1182.0,394.0,0.007614,0.007614,0.002538,7,2.333333,3,1,1.154701
1,1.0,C_ID_130fd0cbdd,2,3,0,2017-01-01,2017.0,1.0,52.0,6.0,484.0,772.0,17167.0,968.0,1452.0,0.0,0.004132,0.006198,0.000000,5,1.666667,3,0,1.527525
2,8.0,C_ID_b709037bc5,5,1,1,2017-08-01,2017.0,3.0,31.0,1.0,272.0,560.0,17379.0,1360.0,272.0,272.0,0.018382,0.003676,0.003676,7,2.333333,5,1,2.309401
3,12.0,C_ID_d27d835a9f,2,1,0,2017-12-01,2017.0,4.0,48.0,4.0,150.0,438.0,17501.0,300.0,150.0,0.0,0.013333,0.006667,0.000000,3,1.000000,2,0,1.000000
4,12.0,C_ID_2b5e3df5c2,5,1,1,2015-12-01,2015.0,4.0,49.0,1.0,881.0,1169.0,16770.0,4405.0,881.0,881.0,0.005675,0.001135,0.001135,7,2.333333,5,1,2.309401


In [222]:
for col in num_cols :
    test[col+'_new'] = 0
    card_id_list = train_transaction.loc[train_transaction[col] > train_transaction[col].mean(), 'card_id'].unique()
    train.loc[card_id_list,col+'_new'] = 1
        

,first_active_month,feature_1,feature_2,feature_3,target,authorized_flag_new,installments_new,month_lag_new,purchase_amount_new,target_out
card_id,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,1,1,1,1,0
C_ID_3d0044924f,2017-01,4,1,0,0.392913,1,1,1,1,0
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,1,0,1,0,0
C_ID_186d6a6901,2017-09,4,3,0,0.142495,1,1,1,1,0
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,1,1,1,1,0


In [224]:
for i, (idx_train, idx_cv) in enumerate(cv.split(train)):
    df_train = train.iloc[idx_train]
    df_cv = train.iloc[idx_cv]
    
    x_train = df_train.drop(['target_out','first_active_month'], axis=1)
    y_train = df_train['target_out']
    
    x_cv = df_cv.drop(['target_out','first_active_month'], axis=1)
    y_cv = df_cv['target_out']
    
    lgb.fit(x_train, y_train)
    
    y_train_pred = lgb.predict(x_train)
    y_train_pred[y_train_pred>=0.75] = 1; y_train_pred[y_train_pred<0.75] = 0
    y_cv_pred = lgb.predict(x_cv)
    y_cv_pred[y_cv_pred>=0.75] = 1; y_cv_pred[y_cv_pred<0.75] = 0
    print("[K = {}] train f1 = {}, cv f1= {}".format(i, f1_score(y_train,y_train_pred), f1_score(y_cv,y_cv_pred)))

[K = 0] train f1 = 1.0, cv f1= 1.0
[K = 1] train f1 = 1.0, cv f1= 1.0
[K = 2] train f1 = 1.0, cv f1= 1.0
[K = 3] train f1 = 1.0, cv f1= 1.0
[K = 4] train f1 = 1.0, cv f1= 1.0


# test

In [ ]:
#test data에도 똑같이 적용해서 해볼것

In [225]:
test = pd.read_csv(path + 'test.csv')
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [227]:
test.columns

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3'], dtype='object')

In [ ]:
#transaction data 중 test data에 있는 데이터만 뽑음
test_transaction = transaction.loc[transaction['card_id'].isin(test['card_id'].tolist()),:]
test_transaction.head()
del transaction

In [234]:
%who

KFold	 LGBMRegressor	 a	 card	 card_id_list	 cat_cols	 col	 cv	 df_cv	 
df_train	 i	 idx_cv	 idx_train	 ip	 jupyternotify	 lgb	 mode	 np	 
num_cols	 path	 pd	 plt	 stats	 test	 test2	 time	 train	 
train_transaction	 x_cv	 x_test	 x_train	 y_cv	 y_cv_pred	 y_test	 y_train	 


In [237]:
del train_transaction	 	 	 	 

In [107]:
x_test = test.drop(['card_id','first_active'], axis = 1)
x_test.shape

(123623, 22)

In [109]:
df_train.shape

(161534, 38)

In [108]:
y_test = lgb.predict(x_test)
y_test[y_test>=0.75] = 1; y_test[y_test<0.75] = 0

ValueError: Number of features of the model must match the input. Model n_features_ is 35 and input n_features is 22 

In [83]:
path = 'C:/Users/wjssm/Desktop/kaggle/competition/Elo_Merchant_Category_Recommendation/'

In [84]:
test2 = pd.read_csv(path + '190205_lgb.csv')

In [86]:
y_test[y_test==1]

array([1., 1., 1., ..., 1., 1., 1.])

In [96]:
x_test.shape

(161534, 35)

In [94]:
y_test.shape

(161534,)

In [95]:
test2.shape

(123623, 2)

In [93]:
pd.concat([test2, pd.DataFrame({'y_test' : y_test})], axis = 1)

,card_id,target,y_test
0,C_ID_0ab67a22ab,-5.288321,0.0
1,C_ID_130fd0cbdd,-0.538509,0.0
2,C_ID_b709037bc5,0.021150,0.0
3,C_ID_d27d835a9f,-0.213703,0.0
4,C_ID_2b5e3df5c2,-0.221106,0.0
5,C_ID_5814b4f13c,0.352866,0.0
6,C_ID_a1b3c75277,-0.225538,0.0
7,C_ID_f7cada36d3,0.371700,0.0
8,C_ID_9d2bc8dfc4,0.057166,0.0
9,C_ID_6d8dba8475,-1.731717,0.0


In [ ]:
#new에 없을수록 -33d으로 예측될 확률이 두배 높다.

In [ ]:
transaction.

In [7]:
for col in transaction.card_id.tolist() :
    if col in target_out.card_id.tolist() :
        transaction.loc[transaction.card_id == col, ]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
transaction.loc[transaction.card_id == target_out.card_id.tolist(), :]

ValueError: Arrays were different lengths: 31075392 vs 2207